
## 目录与步骤指南
**该 notebook 仅供展示，如需运行请访问 github 仓库 https://github.com/dptech-corp/material-compute-agent**

本 Notebook 展示了一个创新的材料计算代理系统，它利用大型语言模型（LLM）和 Google-sdk 多代理框架自动化执行材料科学中的计算任务。系统模拟了真实研究团队的协作过程，包括从文献中提取信息、生成计算输入文件、执行计算、分析结果并生成报告。

该系统特别适用于高通量材料筛选和计算材料科学研究，能够显著提高材料发现和优化的效率。通过集成 Bohrium 计算平台和 VASP 计算软件，系统可以执行从密度泛函理论（DFT）计算到热力学性质分析的各种任务。

本演示特别关注钙钛矿氧化物材料（如 Sr5Ca3Fe8O24）的计算模拟，这类材料在热化学能量存储（TCES）等领域具有重要应用前景。

1. **环境配置**
   - 安装依赖包 (`%pip install -e .`)
   - 下载并配置 MP 数据集
   - 设置 API 密钥 (Bohrium, OpenAI/Azure, DeepSeek)

2. **系统初始化**
   - 导入必要的库和工具
   - 创建模型实例
   - 配置代理角色和工具

3. **任务执行**
   - 启动 MCP 服务 (`python CalculationMCPServer.py`)
   - 连接到 MCP 服务器
   - 创建并配置工作流程
   - 提交计算任务

4. **输入示例**
   - 文章路径: test.pdf
   - 材料体系: Sr5Ca3Fe8O24
   - 参数修改: INCAR 的 NELM = 2


## 配置环境

In [ ]:
%pip install -e .

### 安装SDK

In [ ]:
!git clone -b lh https://github.com/dptech-corp/science-agent-sdk.git
%pip install -e ./science-agent-sdk

### 配置本地mp数据集，用于查询模板

In [ ]:
!wget https://openfiles.mlops.dp.tech/projects/lp/6583cfd69e8f402898b60830889d242d/mp_materials_cif.zip

!unzip -q mp_materials_cif.zip

In [1]:
import os
import nest_asyncio
import asyncio
nest_asyncio.apply()

import subprocess
from contextlib import AsyncExitStack

from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset, SseServerParams
from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.models.lite_llm import LiteLlm
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from dp.agent.adapter.google.client.CalculationToolset import CalculationToolset

from google.adk.tools import FunctionTool

from vasp_function import read_vasp_pdf,write_vasp_report,analyze_vasprun_all,search_poscar_template,write_poscar,check_vasp_input,write_vasp_config
from vasp_function import show_vasp_config,rewrite_vasp_config
from utils import ask_human_for_advice,show_task_status

from sympy import N

### 配置API和运行环境镜像

1. 填入项目，镜像地址，DP access key，OPENAI api
2. 将notebook ip加入lvmeng




In [ ]:
OS = os.uname().sysname

# 安装 bohr CLI
if OS == "Linux":
    print("🌱 正在安装适用于 Linux 的 bohr CLI...")
    subprocess.run("bash -c \"$(curl -fsSL https://dp-public.oss-cn-beijing.aliyuncs.com/bohrctl/1.0.0/install_bohr_linux_curl.sh)\"", shell=True)
elif OS == "Darwin":
    print("🍎 正在安装适用于 macOS 的 bohr CLI...")
    subprocess.run("bash -c \"$(curl -fsSL https://dp-public.oss-cn-beijing.aliyuncs.com/bohrctl/1.0.0/install_bohr_mac_curl.sh)\"", shell=True)
else:
    raise RuntimeError(f"❌ 不支持的操作系统: {OS}")

# 更新PATH环境变量（记得展开~成绝对路径）
home_dir = os.path.expanduser("~")
bohr_path = os.path.join(home_dir, ".bohrium")
os.environ["PATH"] += f":{bohr_path}"

print("✅ bohrctl 安装并配置完成！")

In [ ]:
# 配置 bohrium access key
!echo 'export ACCESS_KEY=YOUR-BOHRIUM-KEY' >> ~/.bashrc #DP access key
!source ~/.bashrc
print("start install bohrctl")
# 配置OpenAI key
os.environ["AZURE_API_BASE"] = "YOUR-OPENAI-URL" #OPENAI api
print("start install bohrctl")
os.environ["AZURE_API_KEY"] = "YOUR-OPENAI-API-KEY" #OPENAI api
os.environ["AZURE_API_VERSION"] = "2024-08-01-preview"#OPENAI api
os.environ["AZURE_DEPLOYMENT_NAME"] = "gpt-4o"

#配置DeepSeek key
os.environ["DEEP_SEEK_BASE_URL"] = "YOUR-DEEPSEEK-URL" #DeepSeek api
os.environ["DEEP_SEEK_API_KEY"] = "YOUR-DEEPSEEK-API-KEY"#DeepSeek api
os.environ["DEEP_SEEK_MODEL_NAME"] = "YOUR-DEEPSEEK-MODEL-NAME" #DeepSeek api


#加入本地数据集位置
os.environ["MP_ROOT_DIR"] = "mp_materials_cif"# 本地数据库位置

In [ ]:
current_dir = os.getcwd()
bohr_executor = {
    "type": "dispatcher",
    "machine": {
        "batch_type": "Bohrium",
        "context_type": "Bohrium",
        "remote_profile": {
            "email": os.environ.get("BOHRIUM_USERNAME"),
            "password": os.environ.get("BOHRIUM_PASSWORD"),
            "program_id": int(os.environ.get("BOHRIUM_PROJECT_ID")),
            "input_data": {
                "image_name": "registry.dp.tech/dptech/vasp:5.4.4",
                "job_type": "container",
                "platform": "ali",
                "scass_type": "c32_m32_cpu",
            },
        },
    },
    "DEFAULT_FORWARD_DIR":[f"{current_dir}/tmp"]
    
}
bohr_storage = {
    "type": "bohrium",
    "username": os.environ.get("BOHRIUM_USERNAME"),
    "password": os.environ.get("BOHRIUM_PASSWORD"),
    "project_id": int(os.environ.get("BOHRIUM_PROJECT_ID")),
}


### 将MCP中的计算工具在这里重新封装，写上注释

In [5]:
async def submit_vasp_job() -> int:

    """

    **you should use the vasp_job after people confirm the poscar file and the vasp_config file.**

    Submit VASP calculation jobs in each subdirectory under the 'tmp' folder.

    This function changes the current working directory to 'tmp', locates all subdirectories,
    and runs the VASP job in each one by executing a shell command. After execution,
    it returns to the original directory.

    Returns:
        dict: A dictionary with 'status' and 'message'. Returns success if all jobs are launched.
              On failure, raises an exception with error details.

    Raises:
        FileNotFoundError: If the 'tmp' directory does not exist or cannot be accessed.
        Exception: If no subdirectories are found or if job execution fails.

    Example:
        result = vasp_job()
        e.g., "All jobs submitted successfully 🎉"
    """

    async with AsyncExitStack() as stack:
        tools, _ = await CalculationToolset.from_server(
            connection_params=SseServerParams(
                url="http://127.0.0.1:8000/sse",
            ),
            default_executor=bohr_executor,
            default_storage=bohr_storage,
            async_exit_stack=stack,
        )

        tool_map = {t.name: t for t in tools}
        result = await tool_map["submit_vasp_job"].run_async(args={}, tool_context=None)
        
    return result


In [6]:
agent_prompt = '''
你是一个集文献阅读、材料建模、VASP 配置、任务执行、结果分析和报告撰写于一体的智能科研助理。

你的整体目标是：根据用户提供的材料体系或文献，**自动生成结构、配置并提交 VASP 任务，最终分析结果并生成一份标准报告**。
---
请你按顺序完成以下任务，不要跳步，不要遗漏任何一步。
请注意，提交任务前一定要等人类反馈！！！
## 🧠 工作总流程如下：

1. **获取任务信息**  
    - 向用户提问：请提供论文路径（PDF）或目标化学式。
    - 使用 `read_vasp_pdf` 工具获取论文内容（无需回复内容，只提取信息）。
    
2. **结构构建与确认**  
    - 使用 `search_poscar_template` 生成 POSCAR 模板。
    - ***
    对POSCAR模板进行原子替换(不再需要search_poscar_template)，确保结构中的所有原子种类、数量和分布都严格符合输入化学式人类希望复现的化学式，请认真完成这最重要的一步。
    比如：
    输入化学式：Sr5Ca3Fe8O24
    人类希望复现的化学式：Sr5Ca3Fe8O24
    那么POSCAR中应该包含Sr,Ca,Fe,O四种原子，且原子数量分别为5,3,8,24，下面的坐标需要根据化学式进行替换，确保符合晶体结构，且元素数目与化学式严格一致。
    ***。
    - 使用ask_human_for_advice 向用户询问原子替换后 POSCAR 文件内容如下,再根据用户反馈进行修改，直到用户满意为止，再进行ti：
      ```
      原子替换后 完整POSCAR 文件内容如下：
      [内容]

      请问你有什么修改意见？
      ```
    - 等待用户确认并接收修改建议。

3. **生成计算配置并检查**  
    - 调用 `write_poscar` 写入 POSCAR，注意这一步需要你需要将原子替换后的，完全符合POSCAR格式的str输入到函数中（注意顶行是化学式）。
    - 根据材料体系命名一个路径 `calcdir`，例如 "LaFeO3"
    - 使用 `write_vasp_config` 写入到calcdir中 生成 INCAR, KPOINTS, POTCAR。
    - 若有缺失，重新生成，确保生成成功。


4. **VASP 提交与监听**  
    - 调用 `show_vasp_config` 获取INCAR 文件内容：
    - 使用ask_human_for_advice 将INCAR文件内容展示给用户，并询问用户是否可以提交任务：
      ```
      以下是 INCAR 文件内容：
      [内容]
      是否可以继续提交任务？
      ```

    - 等到用户确认后，使用 `submit_vasp_job` 提交任务，**注意不要擅自主动提前提交任务**，并监听结果，返回 `xml_path`。

5. **结果分析与报告撰写**  
    - 使用 `analyze_vasprun_all(xml_path)` 分析任务结果。
    - 生成一份标准化报告，内容包括：
      - 程序与平台信息  
      - INCAR 设置摘要  
      - K 点设置与自动化情况  
      - 结构、力、错误信息  
      - 其他重要输出
    - 使用 `write_vasp_report(report_str)` 写入文件。

---

## ⚠️ 交互规则（必须遵守）：

- 每一个阶段都使用**ReAct模式**：先说明你要做什么（Reason），再执行（Action）
- 运行每一步之后，输出结果并使用`ask_human_for_advice`**请求用户确认**是否继续。
- 若中间出错，需立即停止并向用户反馈问题原因。
- **所有参数必须来自文献或用户确认**。
- 所有工具（如 `read_vasp_pdf`, `write_poscar`, `analyze_vasprun_all` 等）都作为你可以调用的功能模块。
- 你是一个严谨、高效、清晰的科研助手，禁止跳步或模糊执行。

---

当你准备好执行任务，使用ask_human_for_advice 向用户询问论文路径和目标化学式：
我已理解任务，将逐步引导您完成一次完整的 VASP 计算流程。
请提供论文路径或目标化学式：


'''


In [7]:
read_vasp_pdf = FunctionTool(func=read_vasp_pdf)
write_vasp_report = FunctionTool(func=write_vasp_report)
analyze_vasprun_all = FunctionTool(func=analyze_vasprun_all)
search_poscar_template = FunctionTool(func=search_poscar_template)
write_poscar = FunctionTool(func=write_poscar)
check_vasp_input = FunctionTool(func=check_vasp_input)
write_vasp_config = FunctionTool(func=write_vasp_config)
show_vasp_config = FunctionTool(func=show_vasp_config)
rewrite_vasp_config = FunctionTool(func=rewrite_vasp_config)
ask_human_for_advice = FunctionTool(func=ask_human_for_advice)
submit_vasp_job = FunctionTool(func=submit_vasp_job)
show_task_status = FunctionTool(func=show_task_status)

In [8]:
vasp_agent = LlmAgent(
    model=LiteLlm(model="azure/gpt-4o"), 
    name="vasp_agent",
    description=(
        "A phd who is good at using VASP to calculate the properties of materials."
    ),
    instruction=agent_prompt,
    tools=[show_task_status,ask_human_for_advice,submit_vasp_job,read_vasp_pdf,write_poscar,write_vasp_config,check_vasp_input,show_vasp_config,rewrite_vasp_config,analyze_vasprun_all,write_vasp_report,search_poscar_template],
)

In [ ]:
async def async_main():
    session_service = InMemorySessionService()
    session = session_service.create_session(
        state={},
        app_name="adk_agent_samples",
        user_id="vasp_agent"
    )

    runner = Runner(
        app_name="adk_agent_samples",
        agent=vasp_agent,
        session_service=session_service,
    )

    print("✅ Vasp任务已启动，输入 'exit' 结束对话。\n")

    while True:
        user_input = input("🧑 请输入指令：")  #
        if user_input.lower() in ["exit", "quit"]:
            print("👋 Vasp任务已结束。")
            break

        content = types.Content(role="user", parts=[types.Part(text=user_input)])
        events_async = runner.run_async(
            session_id=session.id,
            user_id=session.user_id,
            new_message=content
        )

        async for event in events_async:
            if event.content and event.content.parts:
                for part in event.content.parts:
                    if part.text:
                        role = event.content.role
                        if role == "user":
                            print(f"🧑 用户：{part.text}")
                        elif role == "model":
                            print(f"🤖 智能体：{part.text}")

if __name__ == "__main__":
    asyncio.run(async_main())

1. 文章是test.pdf,我想研究Sr5Ca3Fe8O24体系的实验（提供文献路径用于读取不同的文献）
2. 非常好，请继续生成/我想研究的体系是Sr5Ca3Fe8O24，请注意原子替换后给我新的POSCAR
3. 我想把INCAR的NELM = 200 修改为 2（修改INCAR意见）